In [60]:
import pandas as pd
import numpy as np
import pickle

import keras
import keras.utils
import spacy

from keras.callbacks import TensorBoard
from keras import optimizers

from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional
from keras.utils import pad_sequences
from keras_preprocessing import sequence

In [61]:
# Load data from csv to pandas dataframe
# train_data = pd.read_csv('processed_train_data.tsv', sep='\t', index_col=0)
# val_data = pd.read_csv('processed_val_data.tsv', sep='\t', index_col=0)
# test_data = pd.read_csv('processed_test_data.tsv', sep='\t', index_col=0)

# APR 5 TODO
    - [ ] finalise experiments set up
    - [ ] run experiments
    - [ ] go back to preprocessing to add dep parse
    - [ ] include dep in experiments set up
    - [ ] 2nd run
    - [ ] metadata unique size doesnt seem to match the number of unique values

In [62]:
# Alternative way to load data - pickle to pandas dataframe
train_data = pd.read_pickle('processed_train_data.p')
val_data = pd.read_pickle('processed_val_data.p')
test_data = pd.read_pickle('processed_test_data.p')

In [63]:
# def format_string2np(string_numpy):
#     """
#     Converts string representation of a numpy array to a numpy array.
#     Necessary because pandas dataframe cannot store numpy arrays.
#
#     :param string_numpy: {str} string representation of an array
#     :return: numpy array
#     """
#     """formatting : Conversion of String List to List
#
#     Args:
#         string_numpy (str)
#     Returns:
#         l (list): list of values
#     """
#     list_values = string_numpy.strip('[]').split(', ')
#     return np.array(list_values).astype(int)

In [64]:
# # reformat dataframe columns from string to numpy arrays
# columns = ['word_id_custom', 'word_id_spacy', 'pos_id', 'pos_id_DEFAULT']
#
# for col in columns:
#     train_data[col] = train_data[col].apply(format_string2np)
#     val_data[col] = val_data[col].apply(format_string2np)
#     test_data[col] = test_data[col].apply(format_string2np)

In [65]:
# Load POS dictionaries
with open('pos_dicts.pickle', 'rb') as f:
    pos_dict_custom, pos_dict_default = pickle.load(f)

In [66]:
# Load embedding matrixes
embedding_matrix_custom_100d = np.load('embedding_matrix_custom_100d.npy')
embedding_matrix_spacy_100d = np.load('embedding_matrix_spacy_100d.npy')
embedding_matrix_custom_300d = np.load('embedding_matrix_custom_300d.npy')
embedding_matrix_spacy_300d = np.load('embedding_matrix_spacy_300d.npy')

print("Shape of the embedding matrixes:", embedding_matrix_custom_100d.shape, embedding_matrix_spacy_300d.shape)

Shape of the embedding matrixes: (9607, 100) (9496, 300)


In [67]:
# Load vocabulary dictionaries
vocabulary_dict_custom = pickle.load(open('vocabulary_statement_custom.p', 'rb'))
vocabulary_dict_spacy = pickle.load(open('vocabulary_statement_spacy.p', 'rb'))
vocab_length = len(vocabulary_dict_custom)

print("Length of the vocabulary dictionary:", vocab_length)

Length of the vocabulary dictionary: 9606


train_data # train_data for where state_id max is 50

In [68]:
train_data # train_data for where state_id max is 20

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,job_id,state_id,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,1,1,0,2,"[1, 14, 8, 8, 2, 0, 1, 2, 10, 0, 0, 4, 0, 10]","[16, 5, 11, 11, 0, 7, 16, 0, 12, 7, 7, 1, 7, 12]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,6,7,1,2,"[7, 1, 14, 0, 4, 0, 0, 10, 5, 1, 7, 2, 0, 1, 4...","[14, 16, 5, 7, 1, 7, 7, 12, 10, 16, 14, 0, 7, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,2,3,1,8,"[8, 8, 1, 4, 8, 8, 10, 4, 1, 12, 1, 8, 8, 14, ...","[11, 11, 16, 1, 11, 11, 12, 1, 16, 9, 16, 11, ...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,0,0,2,0,"[0, 0, 0, 0, 11, 2, 12, 1, 2, 0, 0, 0, 10]","[7, 7, 7, 7, 3, 0, 9, 16, 0, 7, 7, 7, 12]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,0,2,1,1,"[14, 2, 0, 1, 4, 14, 0, 4, 5, 0, 10]","[5, 0, 7, 16, 1, 5, 7, 1, 10, 7, 12]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,,Florida,none,0.0,...,0,2,2,1,"[5, 1, 14, 2, 0, 4, 0, 0, 4, 8, 7, 5, 1, 0, 4,...","[10, 16, 5, 0, 7, 1, 7, 7, 1, 11, 14, 10, 16, ...",be large number shark attack florida than be c...,large number shark attack florida case voter f...,"[1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...","[126, 100, 5078, 257, 59, 344, 168, 482]"
10236,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,,Georgia,republican,0.0,...,0,7,0,0,"[8, 11, 3, 1, 14, 0, 4, 14, 14, 8, 10, 8, 0, 1...","[11, 3, 2, 16, 5, 7, 1, 5, 13, 11, 12, 11, 7, ...",democrats have now become party atlanta metro ...,democrats party atlanta metro area black,"[157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]","[122, 169, 397, 1298, 574, 325]"
10237,10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,,Georgia,republican,4.0,...,0,7,0,7,"[1, 14, 0, 4, 8, 8, 5, 1, 4, 8, 8, 10, 8, 10, ...","[16, 5, 7, 1, 11, 11, 10, 16, 1, 11, 11, 12, 1...",say alternative social security operate galves...,say alternative social security operate galves...,"[3, 2024, 154, 110, 1205, 3983, 124, 38, 2, 42...","[1, 1919, 119, 80, 1110, 3862, 93, 26, 360, 52..."
10238,10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,,Florida,democrat,3.0,...,0,2,1,7,"[4, 1, 14, 8, 2, 0, 14, 1, 0, 4, 8, 10]","[1, 16, 5, 11, 0, 7, 16, 16, 7, 1, 11, 1

In [6]:
# print(train_data['party_id'].unique())

[ 0  1  2  3  4  8  7 10  5]


## to test the different preprocessing methods, the following experiments will be run:
1. siddarth - baseline

        - ogdict
        - nltk stopword removal only

****
****

## todo
****
### INITIALISE INPUT/OUTPUT
1. [x] word_id custom AS X_train_custom
  - [x] embedding_matrix_custom
  - [x] vocabulary_dict_custom
2. [x] word_id spacy AS X_train_spacy
  - [x] embedding_matrix_spacy
  - [x] vocabulary_dict_spacy
  
3. [x] pos_id custom AS X_train_pos_custom
4. [x] pos_id spacy AS X_train_pos_spacy
    
#### Input variables to be processed: 
- [x] meta
- [ ] dep parse

****
### GENERAL
- [x] change everything to python 3

### variables, init, etc.
- [x] pass in vocabulary.p
****
### BILSTM MODEL
- [x] functions: train(), etc
        (CODE CELLS COULD BE BETTER, INVESTIGATE)
- [ ] verify varibales are correct

****
*decide which word id to use*
- [x] jaccard similarity

1. fathan - pos tag check [ ]
    - custom pos tag dict
    - spacy preprocess - spacy word id
    - glove
 2. fathan - preprocess [ ]
    - custom pos tag dict
    - custom preprocess - custom word id
    - glove
    * check best results with default pos tag [ ]
****

# HYPERPARAMS

In [59]:
# num_state = len(train_data['state_id'].unique())
# print(num_state)
# state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=num_state)

18


IndexError: index 20 is out of bounds for axis 1 with size 18

In [100]:
# 100d GLOVE
EMBED_DIM = 100

# vocab_length = len(vocabulary_dict.keys())
custom_vocabLen = len(vocabulary_dict_custom.keys())
spacy_vocabLen = len(vocabulary_dict_spacy.keys())
hidden_size = EMBED_DIM #Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 30
num_epochs = 30
batch_size = 40

embedding_matrix = embedding_matrix_custom_100d

#Meta data related hyper params
num_party = len(train_data.party_id.unique())
num_state = len(train_data['state_id'].unique())
num_context = len(train_data['context_id'].unique())
num_job = len(train_data['job_id'].unique())
num_sub = len(train_data['subject_id'].unique())
num_speaker = len(train_data['speaker_id'].unique())

print(num_party)
print(num_state)
print(num_context)
print(num_job)
print(num_sub)
print(num_speaker)
print(train_data.columns)

9
18
18
19
25
26
Index(['index', 'id', 'label', 'statement', 'subject', 'speaker', 'job_title',
       'state_info', 'party', 'barely true', 'false', 'half-true',
       'mostly-true', 'pants-on-fire', 'context', 'output', 'subject_id',
       'speaker_id', 'job_id', 'state_id', 'party_id', 'context_id', 'pos_id',
       'pos_id_DEFAULT', 'statement_custom', 'statement_spacy',
       'word_id_custom', 'word_id_spacy'],
      dtype='object')


In [8]:
print(train_data['word_id_custom'])

0        [3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...
1        [37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...
2        [104, 69, 734, 160, 201, 18, 89, 262, 137, 258...
3        [19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...
4                               [282, 3331, 308, 247, 248]
                               ...                        
10235    [1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...
10236       [157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]
10237    [3, 2024, 154, 110, 1205, 3983, 124, 38, 2, 42...
10238       [1604, 28, 13, 2857, 3115, 4, 153, 1391, 1473]
10239    [216, 1429, 2087, 2, 5142, 826, 494, 327, 441,...
Name: word_id_custom, Length: 10240, dtype: object


In [9]:
print(train_data['party_id'])

0        0
1        1
2        1
3        2
4        1
        ..
10235    2
10236    0
10237    0
10238    1
10239    0
Name: party_id, Length: 10240, dtype: int64


# Input/Output Processing

In [63]:
# party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=11)
# state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=51)
# context_train = keras.utils.to_categorical(train_data['context_id'], num_classes=21)

In [57]:
# party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=11)
# state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=21)
# context_train = keras.utils.to_categorical(train_data['context_id'], num_classes=21)

## Apr 6 TODO:
    - [ ] fix the input shape for the meta data, as num_party is 11, not 9.
            - num_party = len(train_data.party_id.unique()) returns 9 instead of 11.
            - or is it expecting 11 wrongly and party_id number of labels should be

In [25]:

#Meta data preparation
party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=num_party)
state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=num_state)
context_train = keras.utils.to_categorical(train_data['context_id'], num_classes=num_context)
job_train = keras.utils.to_categorical(train_data['job_id'], num_classes=num_job)
subject_train = keras.utils.to_categorical(train_data['subject_id'], num_classes=num_sub)
speaker_train = keras.utils.to_categorical(train_data['speaker_id'], num_classes=num_speaker)


party_val = keras.utils.to_categorical(val_data['party_id'], num_classes=num_party)
state_val = keras.utils.to_categorical(val_data['state_id'], num_classes=num_state)
context_val = keras.utils.to_categorical(val_data['context_id'], num_classes=num_context)
job_val = keras.utils.to_categorical(val_data['job_id'], num_classes=num_job)
subject_val = keras.utils.to_categorical(val_data['subject_id'], num_classes=num_sub)
speaker_val = keras.utils.to_categorical(val_data['speaker_id'], num_classes=num_speaker)


party_test = keras.utils.to_categorical(test_data['party_id'], num_classes=num_party)
state_test = keras.utils.to_categorical(test_data['state_id'], num_classes=num_state)
context_test = keras.utils.to_categorical(test_data['context_id'], num_classes=num_context)
job_test = keras.utils.to_categorical(test_data['job_id'], num_classes=num_job)
subject_test = keras.utils.to_categorical(test_data['subject_id'], num_classes=num_sub)
speaker_test = keras.utils.to_categorical(test_data['speaker_id'], num_classes=num_speaker)

IndexError: index 30 is out of bounds for axis 1 with size 30

In [71]:
def input_pad_sequences(data):
    padded_data = sequence.pad_sequences(data, maxlen=num_steps, padding='post', truncating='post')
    return padded_data

X_train_custom = input_pad_sequences(train_data['word_id_custom'])
X_val_custom = input_pad_sequences(val_data['word_id_custom'])
X_test_custom = input_pad_sequences(test_data['word_id_custom'])

X_train_spacy = input_pad_sequences(train_data['word_id_spacy'])
X_val_spacy = input_pad_sequences(val_data['word_id_spacy'])
X_test_spacy = input_pad_sequences(test_data['word_id_spacy'])

X_train_pos_custom = input_pad_sequences(train_data['pos_id'])
X_val_pos_custom = input_pad_sequences(val_data['pos_id'])
X_test_pos_custom = input_pad_sequences(test_data['pos_id'])

X_train_pos_DEFAULT = input_pad_sequences(train_data['pos_id_DEFAULT'])
X_val_pos_DEFAULT = input_pad_sequences(val_data['pos_id_DEFAULT'])
X_test_pos_DEFAULT = input_pad_sequences(test_data['pos_id_DEFAULT'])

# X_train_meta = np.hstack((party_train,
#                           state_train,
#                           context_train,
#                           job_train,
#                           subject_train,
#                           speaker_train))
#
# X_val_meta = np.hstack((party_val,
#                         state_val,
#                         context_val,
#                         job_val,
#                         subject_val,
#                         speaker_val))
#
# X_test_meta = np.hstack((party_test,
#                          state_test,
#                          context_test,
#                          job_test,
#                          subject_test,
#                          speaker_test))

#TODO: preprocess dependency parse
# X_train_dep = input_pad_sequences(train_data['dep_id'])
# X_val_dep = input_pad_sequences(val_data['dep_id'])
# X_test_dep = input_pad_sequences(test_data['dep_id'])

In [98]:
print(X_train_pos_custom)

[[ 1 14  8 ...  0  0  0]
 [ 7  1 14 ...  0  0  0]
 [ 8  8  1 ...  0  0  0]
 ...
 [ 1 14  0 ...  0  0  0]
 [ 4  1 14 ...  0  0  0]
 [14  8  4 ...  0  0  0]]


In [73]:
# Initialise input/output

## OUTPUT
Y_train = train_data['output']
Y_train = keras.utils.to_categorical(Y_train, num_classes=6)

Y_val = val_data['output']
Y_val = keras.utils.to_categorical(Y_val, num_classes=6)

In [74]:
print(X_test_custom)
print(X_test_spacy)
print(X_test_pos_custom)
print(X_test_pos_DEFAULT)

[[ 350  425   28 ...    0    0    0]
 [  64    1 1930 ...    0    0    0]
 [   3  160  201 ...    0    0    0]
 ...
 [ 472 1547  191 ...    0    0    0]
 [   3 1406  928 ...    0    0    0]
 [   3   93    1 ...    0    0    0]]
[[ 291  358   19 ...    0    0    0]
 [  45 1829  234 ...    0    0    0]
 [   1  125  157 ...    0    0    0]
 ...
 [ 405 1448  153 ...    0    0    0]
 [   1 1306  836 ...    0    0    0]
 [   1   64   34 ...    0    0    0]]
[[ 1 14  0 ...  0  0  0]
 [ 8 11  4 ...  0  0  0]
 [ 1  8  8 ...  0  0  0]
 ...
 [ 4 14  2 ...  0  0  0]
 [ 1 14  8 ...  0  0  0]
 [ 1 14  0 ...  0  0  0]]
[[16  5  7 ...  0  0  0]
 [11  3  1 ...  0  0  0]
 [16 11 11 ...  0  0  0]
 ...
 [ 1  5  0 ...  0  0  0]
 [16  5 11 ...  0  0  0]
 [16  5  7 ...  0  0  0]]


In [14]:
print(X_train_custom.shape, X_val_custom.shape, X_test_custom.shape)
print(X_train_spacy.shape, X_val_spacy.shape, X_test_spacy.shape)
print(X_train_pos_custom.shape, X_val_pos_custom.shape, X_test_pos_custom.shape)
print(X_train_pos_DEFAULT.shape, X_val_pos_DEFAULT.shape, X_test_pos_DEFAULT.shape)
print(Y_train.shape, Y_val.shape)

(10240, 15) (1284, 15) (1267, 15)
(10240, 15) (1284, 15) (1267, 15)
(10240, 15) (1284, 15) (1267, 15)
(10240, 15) (1284, 15) (1267, 15)
(10240, 6) (1284, 6)


#### 1. fathan - pos tag check [ ]
    - custom word id [ ]
    - custom pos id [ ]
    - glove [ ]

In [75]:
max_value = max(np.max(X_train_pos_custom), np.max(X_val_pos_custom))
print("Maximum value in the input data:", max_value)


Maximum value in the input data: 14


In [76]:
def train(model, name, train_inputs, val_inputs):
    """
    Trains the given model with the provided input tensors.

    :param model: {tf.keras.Model} model to be trained
    :param name: {str} model name used for saving best weights
    :param train_inputs: {dict} dictionary containing training input tensors
    :param val_inputs: {dict} dictionary containing validation input tensors
    """
    sgd = optimizers.SGD(learning_rate=0.025, clipvalue=0.3, nesterov=True)
    adam = optimizers.Adam(learning_rate=0.000075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    tb = TensorBoard()
    csv_logger = keras.callbacks.CSVLogger('training.log')
    filepath = name + "_weights_best.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                                  verbose=1, save_best_only=True, mode='max')

    # model.fit(
    #     inputs,
    #     {'main_output': Y_train},
    #     epochs=num_epochs,
    #     batch_size=batch_size,
    #     validation_data=(
    #         validation_inputs,
    #         {'main_output': Y_val}
    #     ),
    #     callbacks=[tb, csv_logger, checkpoint]
    # )

    model.fit(
        train_inputs,
        {'main_output': Y_train},
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=(
            val_inputs,
            {'main_output': Y_val}
        ),
        callbacks=[tb, csv_logger, checkpoint]
    )


In [16]:
# def compile_model(model):
#     """
#     Compiles the given model with SGD optimizer, categorical_crossentropy loss, and categorical_accuracy metrics.
#
#     :param model: {tf.keras.Model} the model to be compiled
#     """
#     sgd = optimizers.SGD(learning_rate=0.025, clipvalue=0.3, nesterov=True)
#     model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#
# def create_callbacks(name):
#     """
#     Creates a list of callbacks for model training.
#
#     :param name: {str} model name used for saving best weights
#
#     :return: {list} list of callbacks
#     """
#     tb = TensorBoard()
#     csv_logger = keras.callbacks.CSVLogger('training.log')
#     filepath = name + "_weights_best.hdf5"
#     checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
#                                                   verbose=1, save_best_only=True, mode='max')
#     return [tb, csv_logger, checkpoint]

# def merge_dicts(*args, **kwargs):
#     """
#     Merges the input dictionaries from args and kwargs.
#
#     :param *args: {tuple} variable length argument list for input dictionaries
#     :param **kwargs: {dict} arbitrary keyword arguments for input dictionaries
#
#     :return: {dict} merged dictionary
#     """
#     inputs = {}
#     for arg in args:
#         inputs.update(arg)
#     inputs.update(kwargs)
#     return inputs



In [17]:
# def train(model, name, train_inputs, val_inputs):
#     """
#     Trains the given model with the provided input tensors.
#
#     :param model: {tf.keras.Model} model to be trained
#     :param name: {str} model name used for saving best weights
#     :param train_inputs: {dict} dictionary containing training input tensors
#     :param val_inputs: {dict} dictionary containing validation input tensors
#     """
#     compile_model(model)
#     callbacks = create_callbacks(name)
#
#     model.fit(
#         train_inputs,
#         {'main_output': Y_train},
#         epochs=num_epochs,
#         batch_size=batch_size,
#         validation_data=(
#             val_inputs,
#             {'main_output': Y_val}
#         ),
#         callbacks=callbacks
#     )


In [ ]:

# def train(model, name, *args, **kwargs):
#     """
#     Trains the given model with the provided input tensors and keyword arguments.
#
#     :param model: {tf.keras.Model} model to be trained
#     :param name: {str} model name used for saving best weights
#     :param args: {tuple} variable length argument list for input tensors
#     :param kwargs: {dict} keyword arguments for input tensors
#     """
#     compile_model(model)
#     callbacks = create_callbacks(name)
#     inputs = merge_dicts(*args, **kwargs)
#     validation_inputs = merge_dicts(*args, **kwargs)
#
#     model.fit(
#         inputs,
#         {'main_output': Y_train},
#         epochs=num_epochs,
#         batch_size=batch_size,
#         validation_data=(
#             validation_inputs,
#             {'main_output': Y_val}
#         ),
#         callbacks=callbacks
#     )

In [77]:
import operator

def print_best_false_true_predicted(fw, tb):
  sorted_false = sorted(fw.items(), key=operator.itemgetter(1), reverse=True)
  sorted_true = sorted(tb.items(), key=operator.itemgetter(1), reverse=True)
  print('*****************************************************************')
  print('******************** False statements *************************')

  for t in sorted_false[:5]:
    print(t[1])
    print(test_data.loc[t[0]])
    print('=============')
  print('*****************************************************************')
  print('******************** True Statements *************************')
  for t in sorted_true[:5]:
    print(t[1])
    print(test_data.loc[t[0]])
    print('=============')


In [78]:
def load_and_predict(name, inputs):
    """
    Load the model and predict with the provided inputs

    :param name: {str} name of the model to load the model weights
    :param inputs: {dict} dictionary of inputs to be passed to the model

    :return: preds {np.array} array of predicted values/
    """
    model = load_model(name + "_weights_best.hdf5")
    preds = model.predict(inputs,
                          batch_size=batch_size,
                          verbose=1)
    return preds

def calculate_accuracy(predictions, Y_test_gt):
    """
    Calculate the accuracy of the predictions

    :param predictions: {np.array} predicted values
    :param Y_test_gt: {np.array} ground truth values

    :return: accuracy {float} accuracy of the predictions
    """
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == Y_test_gt[i]:
            correct += 1
    accuracy = correct / len(predictions)
    return accuracy

def extract_fw_tb(preds):
    """
    Extracts the worst false predictions and the best true predictions

    :param preds: {np.array} array of predicted values

    :return: false_worst: {dict} dictionary of worst false predictions
    :return: true_best: {dict} dictionary of best true predictions
    """
    false_worst = {}
    true_best = {}

    for p in range(len(preds)):
        if np.argmax(preds[p])==0:
            false_worst[p]=preds[p][0]
        elif np.argmax(preds[p])==5:
            true_best[p]=preds[p][5]

    return false_worst, true_best

def evaluate(model_name, *args, **kwargs):
    """
    Evaluates the given model with provided input tensors

    :param model_name: {str} used to load model weights
    :param args: {list} of dictionaries of inputs
    :param kwargs:

    :return: false_worst: {dict} of the worst false predictions
    :return: true_best: {dict} of the best true predictions
    """
    inputs = {}
    for arg in args:
        inputs.update(arg)
    inputs.update(kwargs)

    preds = load_and_predict(model_name, inputs)

    Y_test_groundtruth = list(test_data['output'])
    predictions = np.array([np.argmax(pred) for pred in preds])

    accuracy = calculate_accuracy(predictions, Y_test_groundtruth)
    print("Correctly Predicted: ", np.sum(predictions == Y_test_groundtruth), "/", len(Y_test_groundtruth))
    print("Accuracy: ", accuracy)

    false_worst, true_best = extract_fw_tb(preds)

    pickle.dump(predictions, open(model_name + "_predictions.p", "wb"))
    return false_worst, true_best



## 

In [ ]:
# def evaluate(name, *args, **kwargs):
#     model = load_model(name + "_weights_best.hdf5")
#     inputs = {}
#     for arg in args:
#         inputs.update(arg)
#     inputs.update(kwargs)
#
#     preds = model.predict(inputs,
#                           batch_size=batch_size,
#                           verbose=1)
#
#     false_worst = {}
#     true_best = {}
#
#     Y_test_gt = list(test_data['output'])
#     predictions = np.array([np.argmax(pred) for pred in preds])
#
#     for p in range(len(preds)):
#         if np.argmax(preds[p])==0:
#             false_worst[p]=preds[p][0]
#         elif np.argmax(preds[p])==5:
#             true_best[p]=preds[p][5]
#
#     # print(len(predictions))==len(Y_test_gt)
#     correct = np.sum(predictions == Y_test_gt)
#     print("Correctly Predicted: ", correct,"/",len(Y_test_gt))
#     print("Accuracy: ", correct*100.0/len(Y_test_gt))
#
#     pickle.dump(predictions, open(name+"_predictions.p", "wb"))
#     return false_worst, true_best

configurations = [
	{
        "name": "fathan_1",
        "inputs": [X_test_custom, X_test_pos],
    },
	{
        "name": "fathan_2",
        "inputs": [X_test_spacy, X_test_pos],
    },
	{
        "name": "fathan_3",
        "inputs": [X_test_custom, X_test_pos_DEFAULT],
    },
    {
        "name": "fathan_4",
        "inputs": [X_test_custom, X_test_pos_DEFAULT],
    },
	# {
    #     "name": "fathan_5",
    #     "inputs": [X_test_best, X_test_pos_best, X_test_meta],
    # },
	# {
    #     "name": "fathan_6",
    #     "inputs": [X_test_best, X_test_pos_best, X_test_meta, X_test_dep],
    # },
]



In [ ]:
# model_name = "bilstm"
#
# for config in configurations:
#     print(f"Running {config['name']}...")
#     (fw, tb) = evaluate(model_name, *config["inputs"])


In [20]:
train_data

,index,id,label,statement,subject,speaker,job_title,state_info,party,barely true,...,job_id,state_id,party_id,context_id,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,...,1,1,0,2,"[1, 14, 8, 8, 2, 0, 1, 2, 10, 0, 0, 4, 0, 10]","[16, 5, 11, 11, 0, 7, 16, 0, 12, 7, 7, 1, 7, 12]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]"
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,...,6,7,1,2,"[7, 1, 14, 0, 4, 0, 0, 10, 5, 1, 7, 2, 0, 1, 4...","[14, 16, 5, 7, 1, 7, 7, 12, 10, 16, 14, 0, 7, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1..."
2,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,...,2,3,1,8,"[8, 8, 1, 4, 8, 8, 10, 4, 1, 12, 1, 8, 8, 14, ...","[11, 11, 16, 1, 11, 11, 12, 1, 16, 9, 16, 11, ...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,...,0,0,2,0,"[0, 0, 0, 0, 11, 2, 12, 1, 2, 0, 0, 0, 10]","[7, 7, 7, 7, 3, 0, 9, 16, 0, 7, 7, 7, 12]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]"
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,...,0,2,1,1,"[14, 2, 0, 1, 4, 14, 0, 4, 5, 0, 10]","[5, 0, 7, 16, 1, 5, 7, 1, 10, 7, 12]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,,Florida,none,0.0,...,0,2,2,1,"[5, 1, 14, 2, 0, 4, 0, 0, 4, 8, 7, 5, 1, 0, 4,...","[10, 16, 5, 0, 7, 1, 7, 7, 1, 11, 14, 10, 16, ...",be large number shark attack florida than be c...,large number shark attack florida case voter f...,"[1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...","[126, 100, 5078, 257, 59, 344, 168, 482]"
10236,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,,Georgia,republican,0.0,...,0,7,0,0,"[8, 11, 3, 1, 14, 0, 4, 14, 14, 8, 10, 8, 0, 1...","[11, 3, 2, 16, 5, 7, 1, 5, 13, 11, 12, 11, 7, ...",democrats have now become party atlanta metro ...,democrats party atlanta metro area black,"[157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]","[122, 169, 397, 1298, 574, 325]"
10237,10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,,Georgia,republican,4.0,...,0,7,0,7,"[1, 14, 0, 4, 8, 8, 5, 1, 4, 8, 8, 10, 8, 10, ...","[16, 5, 7, 1, 11, 11, 10, 16, 1, 11, 11, 12, 1...",say alternative social security operate galves...,say alternative social security operate galves...,"[3, 2024, 154, 110, 1205, 3983, 124, 38, 2, 42...","[1, 1919, 119, 80, 1110, 3862, 93, 26, 360, 52..."
10238,10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,,Florida,democrat,3.0,...,0,2,1,7,"[4, 1, 14, 8, 2, 0, 14, 1, 0, 4, 8, 10]","[1, 16, 5, 11, 0, 7, 16, 16, 7, 1, 11, 1

In [99]:
model_bilstm = Sequential()
model_bilstm.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
model_bilstm.add(Bidirectional(LSTM(hidden_size)))
model_bilstm.add(Dense(6, activation='softmax'))
# model_bilstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# statement embed bilstm
statement_input = Input(shape=num_steps, dtype='int32', name='main_input')
x = Embedding(vocab_length+1,EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=False)(statement_input)
bilstm_word_input = LSTM(lstm_size, dropout=0.2)(x)

# SPECIFY POS_DICT USED FOR POS EMBEDDING
pos_dict = pos_dict_custom
# pos_embeddings = np.identity(len(set(pos_dict.values())), dtype=int)
pos_embeddings = np.identity(max(pos_dict.values())+1, dtype=int)

# pos embed bilstm
pos_input = Input(shape=num_steps, dtype='int32', name='pos_input')
x2 = Embedding(max(pos_dict.values())+1, max(pos_dict.values())+1, weights=[pos_embeddings], input_length=num_steps, trainable=False)(pos_input)
# x2 = Embedding(len(set(pos_dict.values())), len(set(pos_dict.values())), weights=[pos_embeddings], input_length=num_steps, trainable=False)(pos_input)
bilstm_pos_in = LSTM(lstm_size, dropout=0.2)(x2)

# todo: DEP PARSE
# # dep embed LSTM X3
# dep_input = Input(shape=(num_steps,), dtype='int32', name='dep_input')
# x3 = Embedding(max(dep_dict.values()), max(dep_dict.values()), weights=[dep_embeddings], input_length=num_steps, trainable=False)(dep_input)
# lstm_in3 = LSTM(lstm_size, dropout=0.2)(x3)

# meta data Dense layer
# meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
# x_meta = Dense(64, activation='relu')(meta_input)

ValueError: Please initialize `Bidirectional` layer with a `tf.keras.layers.Layer` instance. Received: 100

In [80]:
model_bilstm.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 50, 100)           960700    
                                                                 
 bidirectional_9 (Bidirectio  (None, 200)              160800    
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 6)                 1206      
                                                                 
Total params: 1,122,706
Trainable params: 1,122,706
Non-trainable params: 0
_________________________________________________________________


In [57]:
model_bilstm.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 20, 100)           960700    
                                                                 
 bidirectional_8 (Bidirectio  (None, 200)              160800    
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 6)                 1206      
                                                                 
Total params: 1,122,706
Trainable params: 1,122,706
Non-trainable params: 0
_________________________________________________________________


In [42]:
print(max(pos_dict.values()))

14


In [81]:
def concat_lstm_layers(bilstm_input, layers_to_concat):
    layers = [bilstm_input] + layers_to_concat
    x = keras.layers.concatenate(layers)
    return x


# extra_layers_for_pos_dep_meta = [bilstm_pos_in, bilstm_dep_in, x_meta]

#, lstm_in3, x_meta]
# exp1_layers = [bilstm_pos_in, lstm_in3, x_meta]
# exp2_layers = [bilstm_pos_in, x_meta]


## Train

In [93]:
# model_lstm = Model(inputs=[statement_input, pos_input, dep_input, meta_input], outputs=[main_output])
# model_lstm = Model(inputs=[statement_input, pos_input, meta_input], outputs=[main_output])
# model_lstm = Model(inputs=[statement_input, dep_input, meta_input], outputs=[main_output])
# model_lstm = Model(inputs=[statement_input, meta_input], outputs=[main_output])

extra_layers_for_pos = [bilstm_pos_in]
# extra_layers_for_pos_meta = [bilstm_pos_in, x_meta]
# extra_layeres_for_pos_dep_meta = [bilstm_pos_in, bi_lstm_dep_in, x_meta]

x = concat_lstm_layers(bilstm_word_input, extra_layers_for_pos)
main_output = Dense(6, activation='softmax', name='main_output')(x)

experiment1input = [statement_input, pos_input]
# experiment2input = [statement_input, pos_input, meta_input]
# experiment3input = [statement_input, pos_input, meta_input, dep_input]

model_bilstm = Model(inputs=experiment1input, outputs=[main_output])

In [97]:
model_bilstm.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 50)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 embedding_36 (Embedding)       (None, 50, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_37 (Embedding)       (None, 50, 15)       225         ['pos_input[0][0]']              
                                                                                            

In [83]:
model_bilstm.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 50)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 embedding_25 (Embedding)       (None, 50, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_26 (Embedding)       (None, 50, 14)       196         ['pos_input[0][0]']              
                                                                                            

In [59]:
model_bilstm.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 20)]         0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, 20)]         0           []                               
                                                                                                  
 embedding_22 (Embedding)       (None, 20, 100)      960700      ['main_input[0][0]']             
                                                                                                  
 embedding_23 (Embedding)       (None, 20, 14)       196         ['pos_input[0][0]']              
                                                                                            

### CONFIG 1

In [95]:
vocab_length = len(vocabulary_dict_custom)

print("Shape of the embedding matrixes:", embedding_matrix_custom_100d.shape)
print("Length of the vocabulary dictionary:", vocab_length)
print("X_train_custom shape:", X_train_custom.shape)
print("X_val_custom shape:", X_val_custom.shape)

Shape of the embedding matrixes: (9607, 100)
Length of the vocabulary dictionary: 9606
X_train_custom shape: (10240, 50)
X_val_custom shape: (1284, 50)


In [47]:
vocab_length = len(vocabulary_dict_custom)

print("Shape of the embedding matrixes:", embedding_matrix_custom_100d.shape)
print("Length of the vocabulary dictionary:", vocab_length)
print("X_train_custom shape:", X_train_custom.shape)
print("X_val_custom shape:", X_val_custom.shape)

Shape of the embedding matrixes: (9607, 100)
Length of the vocabulary dictionary: 9606
X_train_custom shape: (10240, 14)
X_val_custom shape: (1284, 14)


In [85]:
print("Max index in X_train_custom:", np.max(X_train_custom))
print("Max index in X_val_custom:", np.max(X_val_custom))


Max index in X_train_custom: 9606
Max index in X_val_custom: 9545


In [27]:
word_index = {word: i+1 for i, word in enumerate(vocabulary_dict_custom)}

In [29]:
def decode_sequence(sequence, index_to_word):
    return " ".join([index_to_word.get(idx, "") for idx in sequence])

index_to_word = {i: word for word, i in word_index.items()}

# Print a decoded sample from X_train_custom
sample_idx = 89  # You can change this to any valid index
print(decode_sequence(X_train_custom[sample_idx], index_to_word))

# Print a decoded sample from X_val_custom
sample_idx = 2  # You can change this to any valid index
print(decode_sequence(X_val_custom[sample_idx], index_to_word))


austin be burden fast grow tax increase major city nation     
say have organization parade be social welfare organization and then be involve political combat back


In [96]:
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )

Epoch 1/30
255/256 [============================>.] - ETA: 0s - loss: 1.6486 - categorical_accuracy: 0.2455
Epoch 1: val_categorical_accuracy improved from -inf to 0.22274, saving model to bilstm_weights_best.hdf5
256/256 [==============================] - 13s 32ms/step - loss: 1.6485 - categorical_accuracy: 0.2452 - val_loss: 1.6248 - val_categorical_accuracy: 0.2227
Epoch 2/30
255/256 [============================>.] - ETA: 0s - loss: 1.6100 - categorical_accuracy: 0.2455
Epoch 2: val_categorical_accuracy did not improve from 0.22274
256/256 [==============================] - 7s 29ms/step - loss: 1.6101 - categorical_accuracy: 0.2452 - val_loss: 1.6167 - val_categorical_accuracy: 0.2220
Epoch 3/30
255/256 [============================>.] - ETA: 0s - loss: 1.6063 - categorical_accuracy: 0.2454
Epoch 3: val_categorical_accuracy did not improve from 0.22274
256/256 [==============================] - 7s 29ms/step - loss: 1.6062 - categorical_accuracy: 0.2455 - val_loss: 1.6155 - val_cate

In [79]:
### TEST 4: remove pos from equation completely
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom},
      {'main_input': X_val_custom}
     )

Epoch 1/30


C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: in user code:

    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\engine\input_spec.py", line 197, in assert_input_compatibilit

In [78]:
### TEST 3: attempting to fix test 2:
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )

Epoch 1/30


C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/embedding_14/embedding_lookup' defined at (most recent call last):
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\fatha\anaconda3\envs\fyp\l

In [66]:
## TEST 2: ERROR = size of pos embeddings
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom}
     )


C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'model/embedding_2/embedding_lookup' defined at (most recent call last):
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\fatha\anaconda3\envs\fyp\lib\

In [45]:
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      validation_data=(
	      {'main_input': X_val_custom, 'pos_input': X_val_pos_custom},
      )
      )

C:\Users\fatha\anaconda3\envs\fyp\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: Data cardinality is ambiguous:
  x sizes: 10240, 10240, 1284, 1284
  y sizes: 10240
Make sure all arrays contain the same number of samples.

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_custom,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 2

In [ ]:
# CONFIG 2
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_custom},
      validation_data=(
	      {'main_input': X_val_spacy, 'pos_input': X_val_pos_custom},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_custom,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 3

In [ ]:
# CONFIG 3
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_DEFAULT},
      validation_data=(
          {'main_input': X_val_custom, 'pos_input': X_val_pos_DEFAULT},
      )
      )


In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_DEFAULT,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 4

In [ ]:
# CONFIG 4
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_DEFAULT},
      validation_data=(
          {'main_input': X_val_spacy, 'pos_input': X_val_pos_DEFAULT},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_DEFAULT,
                    )

print_best_false_true_predicted(fw, tb)

### CONFIG 5

In [ ]:
# CONFIG 5
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_custom},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_custom, 'pos_input': X_val_pos_custom},
          {'aux_input': X_val_meta},
      )
      )

In [17]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_custom,
                    X_test_meta,
                    # X_test_dep,
                    )

print_best_false_true_predicted(fw, tb)

NameError: name 'evaluate' is not defined

#### CONFIG 6

In [ ]:
# CONFIG 6
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_custom},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_spacy, 'pos_input': X_val_pos_custom},
          {'aux_input': X_val_meta},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_custom,
                    X_test_meta,
                    # X_test_dep,
                    )

print_best_false_true_predicted(fw, tb)

# CONFIG 7


In [ ]:
# CONFIG 7
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_custom, 'pos_input': X_train_pos_DEFAULT},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_custom, 'pos_input': X_val_pos_DEFAULT},
          {'aux_input': X_val_meta},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_custom,
                    X_test_pos_DEFAULT,
                    X_test_meta,
                    # X_test_dep,
                    )
print_best_false_true_predicted(fw, tb)

### CONFIG 8

In [ ]:
# CONFIG 8
train(model_bilstm,
      'bilstm',
      {'main_input': X_train_spacy, 'pos_input': X_train_pos_DEFAULT},
      {'aux_input': X_train_meta},
      validation_data=(
          {'main_input': X_val_spacy, 'pos_input': X_val_pos_DEFAULT},
          {'aux_input': X_val_meta},
      )
      )

In [ ]:
(fw, tb) = evaluate('bilstm',
                    X_test_spacy,
                    X_test_pos_DEFAULT,
                    X_test_meta,
                    # X_test_dep,
                    )
print_best_false_true_predicted(fw, tb)

In [ ]:
# train(model, name, {'main_input': X_train, 'pos_input': X_train_pos}, {'aux_input': X_train_meta})
# train(model, name, main_input=X_train, pos_input=X_train_pos, aux_input=X_train_meta)

In [ ]:
# def train(model, name, use_pos=False, use_meta=False, use_dep=False):
#     sgd = optimizers.SGD(lr=0.025, clipvalue=0.3, nesterov=True)
#     adam = optimizers.Adam(lr=0.000075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#     model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     tb = TensorBoard()
#     csv_logger = ker
#     as.callbacks.CSVLogger('training.log')
#     filepath = name + "_weights_best.hdf5"
#     checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
#
#     if use_pos and use_meta:
#         if use_dep:
#             model.fit(
#                 {'main_input': X_train, 'pos_input': X_train_pos, 'aux_input': X_train_meta, 'dep_input': X_train_dep},
#                 {'main_output': Y_train},
#                 epochs=num_epochs,
#                 batch_size=batch_size,
#                 validation_data=(
#                     {'main_input': X_val, 'pos_input': X_val_pos, 'aux_input': X_val_meta, 'dep_input': X_val_dep},
#                     {'main_output': Y_val}),
#                 callbacks=[tb, csv_logger, checkpoint])
#         else:
#             model.fit(
#                 {'main_input': X_train, 'pos_input': X_train_pos, 'aux_input': X_train_meta},
#                 {'main_output': Y_train},
#                 epochs=num_epochs,
#                 batch_size=batch_size,
#                 validation_data=(
#                     {'main_input': X_val, 'pos_input': X_val_pos, 'aux_input': X_val_meta},
#                     {'main_output': Y_val}),
#                 callbacks=[tb, csv_logger, checkpoint])
#     elif use_meta:
#         if use_dep:
#             model.fit(
#                 {'main_input': X_train, 'aux_input': X_train_meta, 'dep_input': X_train_dep},
#                 {'main_output': Y_train},
#                 epochs=num_epochs,
#                 batch_size=batch_size,
#                 validation_data=(
#                     {'main_input': X_val, 'aux_input': X_val_meta, 'dep_input': X_val_dep},
#                     {'main_output': Y_val}),
#                 callbacks=[tb, csv_logger, checkpoint])
#         else:
#             model.fit(
#                 {'main_input': X_train, 'aux_input': X_train_meta},
#                 {'main_output': Y_train},
#                 epochs=num_epochs,
#                 batch_size=batch_size,
#                 validation_data=(
#                     {'main_input': X_val, 'aux_input': X_val_meta},
#                     {'main_output': Y_val}),
#                 callbacks=[tb, csv_logger, checkpoint])
#     elif use_pos:
#         if use_dep:
#             model.fit(
#                 {'main_input': X_train, 'pos_input': X_train_pos, 'dep_input': X_train_dep},
#                 {'main_output': Y_train},
#                 epochs=num_epochs,
#                 batch_size=batch_size,
#                 validation_data=(
#                     {'main_input': X_val, 'pos_input': X_val_pos, 'dep_input': X_val_dep},
#                     {'main_output': Y_val}),
#                 callbacks=[tb, csv_logger, checkpoint])
#         else:
#             model.fit(
#                 {'main_input': X_train, 'pos_input': X_train_pos},
#                 {'main_output': Y_train},
#                 epochs=num_epochs,
#                 batch_size=batch_size,
#                 validation_data=(
#                     {'main_input': X_val, 'pos_input': X_val_pos},
#                     {'main_output': Y_val}),
#                 callbacks=[tb, csv_logger, checkpoint])
#     else:
#         if use_dep:
#             model.fit(
#             {'main_input': X_train,'dep_input':X_train_dep},
#             {'main_output': Y_train}, epochs = num_epochs, batch_size = batch_size,
#             validation_data = (
#                                 {'main_input': X_val, 'dep_input':X_val_dep},
#                                 {'main_output': Y_val}
#                                 ), callbacks=[tb,csv_logger,checkpoint])
#     else:
#       model.fit(
#         {'main_input': X_train},
#         {'main_output': Y_train}, epochs = num_epochs, batch_size = batch_size,
#         validation_data = (
#             {'main_input': X_val},
#             {'main_output': Y_val}
#         ), callbacks=[tb,csv_logger,checkpoint])
#

In [ ]:
# use_pos=False
# use_meta=True
# use_dep=True
#
# # LSTM model
# model_lstm = Sequential()
# model_lstm.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
# model_lstm.add(Bidirectional(LSTM(hidden_size)))
# model_lstm.add(Dense(6, activation='softmax'))
#
#
# # statement embed LSTM
# statement_input = Input(shape=(num_steps,), dtype='int32', name='main_input')
# x = Embedding(vocab_length+1,EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=False)(statement_input)
# lstm_in = LSTM(lstm_size,dropout=0.2)(x)
#
#
#
# # pos embed LSTM
# pos_input = Input(shape=(num_steps,), dtype='int32', name='pos_input')
# x2 = Embedding(max(pos_dict.values()), max(pos_dict.values()), weights=[pos_embeddings], input_length=num_steps, trainable=False)(pos_input)
# lstm_in2 = LSTM(lstm_size, dropout=0.2)(x2)
#
#
# # dep embed LSTM
# dep_input = Input(shape=(num_steps,), dtype='int32', name='dep_input')
# x3 = Embedding(max(dep_dict.values()), max(dep_dict.values()), weights=[dep_embeddings], input_length=num_steps, trainable=False)(dep_input)
# lstm_in3 = LSTM(lstm_size, dropout=0.2)(x3)
#
#
# # meta data Dense
# meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
# x_meta = Dense(64, activation='relu')(meta_input)
#
#
# if use_pos and use_meta:
#   if use_dep:
#     x = keras.layers.concatenate([lstm_in, lstm_in2, lstm_in3, x_meta])
#   else:
#     x = keras.layers.concatenate([lstm_in, lstm_in2, x_meta])
# elif use_meta:
#   if use_dep:
#     x = keras.layers.concatenate([lstm_in, lstm_in3, x_meta])
#   else:
#     x = keras.layers.concatenate([lstm_in, x_meta])
# elif use_pos:
#   if use_dep:
#     x = keras.layers.concatenate([lstm_in, lstm_in3, lstm_in2])
#   else:
#     x = keras.layers.concatenate([lstm_in, lstm_in2])
# else:
#   if use_dep:
#     x = keras.layers.concatenate([lstm_in, lstm_in3])
#   else:
#     x = lstm_in
#
#
#
#
# main_output = Dense(6, activation='softmax', name='main_output')(x)
#
# if use_pos and use_meta:
#   if use_dep:
#     model_lstm = Model(inputs=[statement_input, pos_input, dep_input, meta_input], outputs=[main_output])
#   else:
#     model_lstm = Model(inputs=[statement_input, pos_input, meta_input], outputs=[main_output])
# elif use_meta:
#   if use_dep:
#     model_lstm = Model(inputs=[statement_input, dep_input, meta_input], outputs=[main_output])
#   else:
#     model_lstm = Model(inputs=[statement_input, meta_input], outputs=[main_output])
# elif use_pos:
#   if use_dep:
#     model_lstm = Model(inputs=[statement_input, dep_input, pos_input], outputs=[main_output])
#   else:
#     model_lstm = Model(inputs=[statement_input, pos_input], outputs=[main_output])
# else:
#   if use_dep:
#     model_lstm = Model(inputs=[statement_input, dep_input], outputs=[main_output])
#   else:
#     model_lstm = Model(inputs=[statement_input], outputs=[main_output])

In [ ]:
# print model_lstm.summary()